In [1]:
import cv2
from keras.models import load_model
import numpy as np
import os

Using TensorFlow backend.


In [2]:
model=load_model(r'F:\Machine Learning\emojinator.h5')

W0928 19:12:30.589241 14800 deprecation_wrapper.py:119] From E:\software\anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0928 19:12:31.180747 14800 deprecation_wrapper.py:119] From E:\software\anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0928 19:12:31.443678 14800 deprecation_wrapper.py:119] From E:\software\anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0928 19:12:31.770796 14800 deprecation_wrapper.py:119] From E:\software\anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0928 19:12:31.950849 14800 deprecation_wrapper.py:119] From E:\software\anaconda\lib\site-pack

In [3]:
def get_emojis():
    emojis_folder = 'hand_emo/'
    emojis=[]
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png',-1))
    return emojis    

In [4]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

In [5]:
def keras_process_image(img):
    image_x = 50
    image_y = 50
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [6]:
def overlay(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

In [7]:
def blend_transparent(face_img, overlay_t_img):
    overlay_img = overlay_t_img[:,:,:3] 
    overlay_mask = overlay_t_img[:,:,3:] 
    background_mask = 255 - overlay_mask
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))

In [8]:
def main():
    emojis = get_emojis()
    cap = cv2.VideoCapture(0)
    x, y, w, h = 300, 50, 350, 350

    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        if ret:
            hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
            mask2 = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
            res = cv2.bitwise_and(img, img, mask=mask2)
            gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
            median = cv2.GaussianBlur(gray, (5, 5), 0)

            kernel_square = np.ones((5, 5), np.uint8)
            dilation = cv2.dilate(median, kernel_square, iterations=2)
            opening = cv2.morphologyEx(dilation, cv2.MORPH_CLOSE, kernel_square)
            ret, thresh = cv2.threshold(opening, 30, 255, cv2.THRESH_BINARY)
            thresh = thresh[y:y + h, x:x + w]
            contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
            if len(contours) > 0:
                contour = max(contours, key=cv2.contourArea)
                if cv2.contourArea(contour) > 2500:
                    x, y, w1, h1 = cv2.boundingRect(contour)
                    newImage = thresh[y:y + h1, x:x + w1]
                    newImage = cv2.resize(newImage, (50, 50))
                    pred_probab, pred_class = keras_predict(model, newImage)
                    print(pred_class, pred_probab)
                    img = overlay(img, emojis[pred_class], 400, 250, 90, 90)

            x, y, w, h = 300, 50, 350, 350
            cv2.imshow("Frame", img)
            cv2.imshow("Contours", thresh)
            if cv2.waitKey(27) & 0xFF==ord('q'):
                break 


In [ ]:
keras_predict(model, np.zeros((50, 50, 1), dtype=np.uint8))
main()

0
1
2
3
4
5
6
7
8
9
10
11
12
1 0.99613035
11 0.935846
11 0.935846
10 0.77906895
10 0.77906895
10 0.5969667
10 0.5969667
11 0.63247716
11 0.63247716
6 0.9114301
6 0.9114301
6 0.9995765
6 0.9995765
11 0.70888853
11 0.70888853
6 0.5737548
6 0.5737548
11 0.70249724
11 0.70249724
11 0.93967736
11 0.93967736
11 0.98770267
11 0.98770267
11 0.9979231
11 0.9979231
11 0.9969695
11 0.9969695
11 0.99641305
11 0.99641305
11 0.9973955
11 0.9973955
3 0.64194703
3 0.64194703
4 0.6120482
4 0.6120482
6 0.62537444
6 0.62537444
6 0.65365756
6 0.65365756
6 0.6003806
6 0.6003806
4 0.89442927
4 0.89442927
4 0.6918619
4 0.6918619
4 0.86174273
4 0.86174273
4 0.93695086
4 0.93695086
4 0.8370107
4 0.8370107
4 0.6075009
4 0.6075009
4 0.53031343
4 0.53031343
3 0.6049118
3 0.6049118
4 0.97069824
4 0.97069824
6 0.80335754
6 0.80335754
6 0.6660654
6 0.6660654
6 0.99640936
6 0.99640936
6 0.9621314
6 0.9621314
6 0.9964629
6 0.9964629
6 0.9941204
6 0.9941204
6 0.9752584
6 0.9752584
6 0.9700361
6 0.9700361
6 0.81325924
6

3 0.723908
6 0.44663903
6 0.44663903
6 0.48236805
6 0.48236805
6 0.8957225
6 0.8957225
6 0.7658011
6 0.7658011
4 0.5966289
4 0.5966289
9 0.63137656
9 0.63137656
9 0.99998546
9 0.99998546
9 0.995812
9 0.995812
9 0.9998049
9 0.9998049
9 0.99977237
9 0.99977237
9 0.99999857
9 0.99999857
9 0.99998844
9 0.99998844
9 0.99966466
9 0.99966466
9 0.91340846
9 0.91340846
11 0.5302137
11 0.5302137
4 0.87357247
4 0.87357247
10 0.5128597
10 0.5128597
10 0.83415127
10 0.83415127
10 0.7697219
10 0.7697219
9 0.8060615
9 0.8060615
6 0.58819306
6 0.58819306
9 0.7511404
9 0.7511404
10 0.5343508
10 0.5343508
10 0.622968
10 0.622968
9 0.74299115
9 0.74299115
10 0.54281735
10 0.54281735
9 0.8253806
9 0.8253806
9 0.8109339
9 0.8109339
9 0.5238265
9 0.5238265
9 0.9129817
9 0.9129817
9 0.40052074
9 0.40052074
6 0.94221205
6 0.94221205
9 0.8772387
9 0.8772387
6 0.7516777
6 0.7516777
9 0.7991227
9 0.7991227
9 0.7194345
9 0.7194345
9 0.7526466
9 0.7526466
9 0.9465439
9 0.9465439
6 0.4751951
6 0.4751951
6 0.6801941

4 0.9980567
4 0.9866108
4 0.9866108
4 0.99044657
4 0.99044657
4 0.88090324
4 0.88090324
4 0.9935588
4 0.9935588
4 0.96671975
4 0.96671975
4 0.98112166
4 0.98112166
4 0.9992392
4 0.9992392
4 0.9940308
4 0.9940308
4 0.9969476
4 0.9969476
3 0.76506424
3 0.76506424
3 0.9211245
3 0.9211245
6 0.99985385
6 0.99985385
6 0.9996631
6 0.9996631
6 0.9979442
6 0.9979442
6 0.9997224
6 0.9997224
6 0.9987631
6 0.9987631
6 0.99930954
6 0.99930954
6 0.97554964
6 0.97554964
6 0.99283737
6 0.99283737
6 0.9926528
6 0.9926528
3 0.5121447
3 0.5121447
5 0.97593486
5 0.97593486
3 0.6564626
3 0.6564626
11 0.5074411
11 0.5074411
11 0.967534
11 0.967534
11 0.68222284
11 0.68222284
8 0.48858336
8 0.48858336
11 0.8561968
11 0.8561968
4 0.94397795
4 0.94397795
9 0.8659529
9 0.8659529
9 0.709244
9 0.709244
9 0.83002347
9 0.83002347
9 0.8620328
9 0.8620328
9 0.8716792
9 0.8716792
9 0.8229381
9 0.8229381
9 0.8038174
9 0.8038174
3 0.25276354
3 0.25276354
3 0.2589316
3 0.2589316
3 0.3097258
3 0.3097258
6 0.27860448
6 0.2

11 0.9662832
11 0.9662832
3 0.56163067
3 0.56163067
3 0.81006825
3 0.81006825
3 0.49224177
3 0.49224177
3 0.7516468
3 0.7516468
3 0.56344384
3 0.56344384
11 0.9056863
11 0.9056863
4 0.41038924
4 0.41038924
11 0.94710785
11 0.94710785
11 0.893913
11 0.893913
1 0.4623731
1 0.4623731
11 0.7031477
11 0.7031477
11 0.9999491
11 0.9999491
11 0.9999794
11 0.9999794
11 0.99998605
11 0.99998605
11 0.9997677
11 0.9997677
11 0.9958669
11 0.9958669
11 0.9965333
11 0.9965333
11 0.9956388
11 0.9956388
11 0.99859136
11 0.99859136
11 0.99705553
11 0.99705553
11 0.986291
11 0.986291
11 0.9982703
11 0.9982703
11 0.98308647
11 0.98308647
11 0.9987312
11 0.9987312
11 0.99767095
11 0.99767095
11 0.9934685
11 0.9934685
11 0.9920439
11 0.9920439
11 0.98902625
11 0.98902625
11 0.9855214
11 0.9855214
11 0.9883167
11 0.9883167
11 0.9515749
11 0.9515749
1 0.5790266
1 0.5790266
1 0.65343237
1 0.65343237
1 0.60891396
1 0.60891396
1 0.5662442
1 0.5662442
1 0.49715444
1 0.49715444
11 0.39656344
11 0.39656344
11 0.551

6 0.91516757
6 0.87779576
6 0.87779576
6 0.93893784
6 0.93893784
6 0.8779176
6 0.8779176
6 0.84814775
6 0.84814775
6 0.77709484
6 0.77709484
6 0.89498353
6 0.89498353
6 0.9967618
6 0.9967618
6 0.58187044
6 0.58187044
6 0.85599524
6 0.85599524
6 0.87052643
6 0.87052643
6 0.75291735
6 0.75291735
6 0.9999958
6 0.9999958
6 0.99228704
6 0.99228704
6 0.92618495
6 0.92618495
6 0.57199126
6 0.57199126
9 0.9262604
9 0.9262604
9 0.9888127
9 0.9888127
9 0.9998635
9 0.9998635
9 0.999941
9 0.999941
9 0.99736136
9 0.99736136
9 0.97609884
9 0.97609884
9 0.99845564
9 0.99845564
9 0.99995494
9 0.99995494
9 0.9996333
9 0.9996333
9 0.9920439
9 0.9920439
6 0.9664686
6 0.9664686
6 0.9890849
6 0.9890849
9 0.8371906
9 0.8371906
9 0.99999416
9 0.99999416
9 0.98873633
9 0.98873633
9 0.9985819
9 0.9985819
9 0.994962
9 0.994962
9 0.9806803
9 0.9806803
9 0.9980982
9 0.9980982
9 0.9982161
9 0.9982161
9 0.9964909
9 0.9964909
9 0.9995664
9 0.9995664
9 0.9996861
9 0.9996861
9 0.9972422
9 0.9972422
9 0.9998454
9 0.999

...

6 0.5576459
6 0.5576459
6 0.9537499
6 0.9537499
6 0.96775806
6 0.96775806
6 0.97389835
6 0.97389835
6 0.95028377
6 0.95028377
6 0.8667455
6 0.8667455
6 0.98986673
6 0.98986673
3 0.40012038
3 0.40012038
3 0.66570795
3 0.66570795
9 0.82749444
9 0.82749444
6 0.99960643
6 0.99960643
6 0.8796692
6 0.8796692
3 0.5674916
3 0.5674916
6 0.82384765
6 0.82384765
6 0.93164915
6 0.93164915
6 0.9997496
6 0.9997496
6 0.9712252
6 0.9712252
9 0.8336167
9 0.8336167
3 0.832186
3 0.832186
7 0.46183667
7 0.46183667
6 0.98254734
6 0.98254734
3 0.5177357
3 0.5177357
9 0.7691168
9 0.7691168
3 0.36256906
3 0.36256906
11 0.42301866
11 0.42301866
6 0.98766696
6 0.98766696
6 0.7781271
6 0.7781271
3 0.63641727
3 0.63641727
5 0.61334574
5 0.61334574
5 0.5134853
5 0.5134853
3 0.35512128
3 0.35512128
3 0.60331094
3 0.60331094
6 0.53695935
6 0.53695935
3 0.58422285
3 0.58422285
7 0.66597676
7 0.66597676
3 0.50981295
3 0.50981295
6 0.5514001
6 0.5514001
6 0.9085361
6 0.9085361
5 0.7690914
5 0.7690914
5 0.83529395
5 0.8

6 0.63380045
6 0.63380045
6 0.8040167
6 0.8040167
6 0.998027
6 0.998027
6 0.9571615
6 0.9571615
6 0.97136736
6 0.97136736
9 1.0
9 1.0
9 0.9999999
9 0.9999999
9 0.9999944
9 0.9999944
9 1.0
9 1.0
9 1.0
9 1.0
9 0.99274373
9 0.99274373
9 0.98848057
9 0.98848057
9 0.99999714
9 0.99999714
6 0.90443254
6 0.90443254
11 0.7911057
11 0.7911057
2 0.54748887
2 0.54748887
11 0.81309557
11 0.81309557
3 0.45841897
3 0.45841897
4 0.8026974
4 0.8026974
10 0.47355765
10 0.47355765
11 0.67577076
11 0.67577076
3 0.9712423
3 0.9712423
3 0.7134376
3 0.7134376
3 0.62769926
3 0.62769926
6 0.64392245
6 0.64392245
11 0.7091928
11 0.7091928
6 0.9999398
6 0.9999398
9 0.5903491
9 0.5903491
11 0.71347934
11 0.71347934
11 0.7283341
11 0.7283341
11 0.8377214
11 0.8377214
11 0.550744
11 0.550744
11 0.5999498
11 0.5999498
9 0.36020994
9 0.36020994
11 0.46109697
11 0.46109697
6 0.9989511
6 0.9989511
11 0.97726125
11 0.97726125
11 0.85192496
11 0.85192496
11 0.9739464
11 0.9739464
9 0.96894395
9 0.96894395
11 0.61246675


4 0.999361
4 0.9645033
4 0.9645033
4 0.9833356
4 0.9833356
4 0.9950767
4 0.9950767
4 0.9916715
4 0.9916715
4 0.9976441
4 0.9976441
4 0.95517004
4 0.95517004
4 0.9934244
4 0.9934244
4 0.99061674
4 0.99061674
4 0.992486
4 0.992486
4 0.99171716
4 0.99171716
4 0.8936527
4 0.8936527
4 0.8754419
4 0.8754419
4 0.7702519
4 0.7702519
3 0.29532784
3 0.29532784
4 0.7090114
4 0.7090114
4 0.4117413
4 0.4117413
4 0.90893704
4 0.90893704
4 0.9889053
4 0.9889053
4 0.98842686
4 0.98842686
4 0.99725705
4 0.99725705
4 0.9999039
4 0.9999039
4 0.9965784
4 0.9965784
4 0.9997029
4 0.9997029
4 0.9989166
4 0.9989166
4 0.9999665
4 0.9999665
4 0.99938405
4 0.99938405
4 0.9979382
4 0.9979382
4 0.99436164
4 0.99436164
7 0.99770504
7 0.99770504
7 0.87958246
7 0.87958246
7 0.5064205
7 0.5064205
4 0.99910176
4 0.99910176
4 0.9930633
4 0.9930633
4 0.99893004
4 0.99893004
7 0.9677228
7 0.9677228
7 0.9998411
7 0.9998411
7 0.99960524
7 0.99960524
7 0.9993673
7 0.9993673
7 0.8057907
7 0.8057907
4 0.88982207
4 0.88982207
7

10 0.60293436
10 0.60293436
4 0.9392035
4 0.9392035
4 0.97387785
4 0.97387785
4 0.9695481
4 0.9695481
4 0.9838126
4 0.9838126
4 0.99563414
4 0.99563414
4 0.8975541
4 0.8975541
4 0.6290346
4 0.6290346
4 0.93914515
4 0.93914515
4 0.9812182
4 0.9812182
4 0.9974496
4 0.9974496
4 0.97948885
4 0.97948885
4 0.9928612
4 0.9928612
4 0.99205726
4 0.99205726
4 0.97906816
4 0.97906816
4 0.95427054
4 0.95427054
4 0.99755776
4 0.99755776
4 0.92551625
4 0.92551625
4 0.9479067
4 0.9479067
4 0.9829213
4 0.9829213
4 0.9738792
4 0.9738792
4 0.99688065
4 0.99688065
4 0.9928757
4 0.9928757
4 0.995165
4 0.995165
4 0.99453676
4 0.99453676
4 0.998285
4 0.998285
4 0.9989668
4 0.9989668
4 0.99131495
4 0.99131495
4 0.88602626
4 0.88602626
4 0.8830438
4 0.8830438
4 0.95233667
4 0.95233667
4 0.9632876
4 0.9632876
4 0.9836505
4 0.9836505
4 0.9961886
4 0.9961886
4 0.97720486
4 0.97720486
7 0.73598576
7 0.73598576
4 0.9620125
4 0.9620125
7 0.49994123
7 0.49994123
7 0.96213216
7 0.96213216
4 0.98618305
4 0.98618305
4 

5 0.9993193
5 0.9993193
5 0.99956673
5 0.99956673
5 1.0
5 1.0
5 0.99999726
5 0.99999726
7 0.47987914
7 0.47987914
3 0.92686874
3 0.92686874
6 0.72006977
6 0.72006977
6 0.41496122
6 0.41496122
6 0.38536474
6 0.38536474
6 0.25658283
6 0.25658283
6 0.52545285
6 0.52545285
6 0.29439396
6 0.29439396
4 0.9965107
4 0.9965107
4 0.9984328
4 0.9984328
4 0.9936516
4 0.9936516
4 0.99786335
4 0.99786335
4 0.99456644
4 0.99456644
4 0.9976776
4 0.9976776
4 0.9974464
4 0.9974464
4 0.9911806
4 0.9911806
4 0.9932748
4 0.9932748
4 0.9975629
4 0.9975629
4 0.9950517
4 0.9950517
4 0.9992822
4 0.9992822
4 0.99456114
4 0.99456114
4 0.9956363
4 0.9956363
4 0.98522955
4 0.98522955
4 0.9890519
4 0.9890519
4 0.99093693
4 0.99093693
4 0.989301
4 0.989301
4 0.98860884
4 0.98860884
4 0.99445564
4 0.99445564
4 0.99498075
4 0.99498075
4 0.9969771
4 0.9969771
4 0.9932683
4 0.9932683
4 0.9941022
4 0.9941022
4 0.99703884
4 0.99703884
4 0.987948
4 0.987948
4 0.99746037
4 0.99746037
4 0.998801
4 0.998801
4 0.9923834
4 0.99